In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from requests_futures.sessions import FuturesSession
from urllib.parse import urljoin
from collections import namedtuple

In [15]:
SCROLL_PAUSE_TIME = 0.5
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-infobars")
driver = webdriver.Chrome(r'C:\Users\nganh\Downloads\Application_FileSetup\chromedriver.exe')
driver.get('https://careerbuilder.vn')
keyword = driver.find_element_by_name('keyword2')
keyword.clear()
keyword.send_keys('data analyst')
keyword.send_keys(Keys.RETURN)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")


start = time.time()
while True:
    
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

source_code = driver.page_source    
soup = bs(source_code, 'lxml')
nextpage = driver.find_elements_by_link_text('Trang kế tiếp')
nextpage[0].click()

WebDriverException: Message: unknown error: Element <a class="right" href="https://careerbuilder.vn/viec-lam/data-analyst-k-trang-2-vi.html">...</a> is not clickable at point (391, 301). Other element would receive the click: <td>...</td>
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)


In [10]:
jobs = soup.find_all('h3', attrs={'class':'job'})
print(len(jobs))

50


In [3]:

urls = []
for item in job_items:
    url_base = 'https://www.vietnamworks.com'
    urls.append(urljoin(url_base, item.select('h3 a')[0]['href']))

In [4]:
TechDetailed = namedtuple('TechDetailed', 'title, company, description, requirement')
def detail(response):
    soup = bs(response.text, 'lxml')
    title = soup.find('h1', attrs={'class':'job-title'}).text
    description = soup.find('div', attrs={'class':'description'}).text
    if soup.find('img' , attrs={'class':'logo img-responsive'}) != None: 
        company = soup.find('img' , attrs={'class':'logo img-responsive'})['alt']
    elif soup.find('span', attrs={'class':'logo-placeholder'}) != None:
        company = soup.find('span', attrs={'class':'logo-placeholder'})['alt']
    else:
        return 0
    requirement = soup.find('div', attrs={'class':'requirements'}).text

    return TechDetailed(title, company, description, requirement)

In [5]:
session = FuturesSession(max_workers=20)
futures = [session.get(url) for url in urls]

In [6]:
details = [detail(future.result()) for future in futures]
data_dict = {'url': urls, 'title': [detail.title for detail in details], 'company': [detail.company for detail in details], 'description':[detail.description for detail in details], 'requirement':[detail.requirement for detail in details] }

In [7]:
data_frame = pd.DataFrame(data_dict)
end = time.time()

In [8]:
data_frame.to_csv('Vietnamwork-DE.csv')

In [9]:
end - start

37.80498480796814